## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [66]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from nltk.corpus import stopwords
# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [73]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [74]:
# List of English stopwords
stop_words = set(stopwords.words('english'))

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    # Join tokens back into a single string
    return ' '.join(tokens)

# Check for tables in the database
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = convention_cur.fetchall()
print("Tables in the database:", tables)

convention_data = []

# Assuming the table name is 'conventions', query to get the text and party
query = '''
    SELECT text, party FROM conventions
'''

try:
    query_results = convention_cur.execute(query)
    for row in query_results:
        text, party = row
        cleaned_text = clean_and_tokenize(text)
        convention_data.append([cleaned_text, party])
finally:
    # Close the database connection after processing
    convention_db.close()
    


Tables in the database: [('conventions',)]


Let's look at some random entries and see if they look right. 

In [75]:
random.choices(convention_data,k=5)

[['rhetoric needed judge president facts 5 million americans infected covid19 170000 americans died far worst performance nation earth 50 million people filed unemployment year 10 million people going lose health insurance year nearly one six small businesses closed year president hes reelected know happen cases deaths remain far high mom pop business close doors time good working families struggle get yet wealthiest 1 get 10s billions dollars new tax breaks assault affordable care act continue destroyed taking insurance away 20 million people including 15 million people medicaid getting rid protections president obama worked hard get past 100 million people preexisting conditions',
  'Democratic'],
 ['help reform broken criminal justice system joe end private prisons detention centers cash bail school prison pipeline heal soul nation joe biden end hate division trump created stop demonization immigrants coddling white nationalists racist dog whistling religious bigotry ugly attacks wo

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [76]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2378 as features in the model.


In [77]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Initialize an empty dictionary to hold the feature words
    ret_dict = dict()
    
    # Split the text into words
    words = text.split()
    
    # Check each word against the feature words
    for word in words:
        if word in fw:
            ret_dict[word] = True
    
    return ret_dict

# Example usage
text = "quick quick brown fox"
fw = {'quick', 'fox', 'jumps'}
print(conv_features(text, fw))

{'quick': True, 'fox': True}


In [78]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [79]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [80]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [81]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.498


In [82]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Republican frequently mention "china". Democrats most frequent terms are "votes" and "climate".



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [86]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [87]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [90]:
tweet_data = []

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Ensure text is in string format
    if isinstance(text, bytes):
        text = text.decode('utf-8')
    # Lowercase the text
    text = text.lower()
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    # Join tokens back into a single string
    return ' '.join(tokens)

# Process each row in the results
for row in results:
    candidate, party, tweet_text = row
    cleaned_text = clean_and_tokenize(tweet_text)
    tweet_data.append([cleaned_text, party])

# Close the database connection
cong_db.close()


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [91]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [92]:

for tweet, party in tweet_data_sample :
    estimated_party = 'Gotta fill this in'
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: lets make even greater kag httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Gotta fill this in.

Her

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [93]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [94]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0,
                          'Democratic': 0,
                          'Gotta fill this in': 4278}),
             'Democratic': defaultdict(int,
                         {'Republican': 0,
                          'Democratic': 0,
                          'Gotta fill this in': 5724})})

### Reflections

The label "Gotta fill this in" indicates that this section is either a temporary placeholder or an incomplete part of the data gathering process. The figures listed under this label (4278 for Republicans and 5724 for Democrats) are substantial and likely represent some raw or unclassified data that requires further organization or refinement. 